In [46]:
import pandas as pd
import numpy as np

In [47]:
df1 = pd.read_csv('../datasets/tong_hop1.csv')
df2 = pd.read_csv('../datasets/tong_hop.csv')
df2 = df2[~df2.label.isna()]

df2.set_index('PID', inplace=True)
df1.set_index('PID', inplace=True)

df1.index = df1.index.astype(str)
df2.index = df2.index.astype(str)

df2 = df2.drop(columns=['indate_x', 'indate_y', 'HBH', 'HBB']).assign(sheet='d4')
df1.shape, df2.shape

((23121, 18), (11255, 20))

In [48]:
intersec_pid = set(df1.index) & set(df2.index)

In [49]:
df3 = df1[~df1.index.isin(intersec_pid)]
df3 = df3.assign(inyear=2017, label=((df3.alpha + df3.beta) > 0).astype(int))
df3 = df3.drop(columns=['alpha', 'beta'])

In [50]:
df = pd.concat([df3, df2])
# df.to_csv('../datasets/tong_hop2.csv')


In [51]:
# df.reset_index().PID.to_csv("../datasets/pid.csv")

In [52]:
address_df = pd.read_csv('../datasets/address2.csv')
address_df.drop_duplicates(subset='PID', keep='first', inplace=True)
address_df.set_index("PID", inplace=True)
address_df.index = address_df.index.astype(str)
address_df.shape

(26462, 2)

In [269]:
numerical_columns = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC', 'FE', 'FERRITIN', 'HBA1', 'HBA2', 'label']

final_df = df.merge(address_df, how='left', left_index=True, right_index=True)
final_df = final_df[final_df.sex.isin(['M', 'F']) & ~final_df.Age.isna() & ~final_df.sheet.isin(['fe_normal'])]
# final_df = final_df[final_df.sex.isin(['M', 'F']) & ~final_df.Age.isna()]
full_df = final_df[numerical_columns].apply(pd.to_numeric,  errors='coerce', downcast='float').dropna()
# full_df = full_df[~((full_df.MCV.astype(float)>=85) & (full_df.MCH.astype(float)>=28))]
full_df2 = final_df[numerical_columns].apply(pd.to_numeric,  errors='coerce', downcast='float').dropna()
full_df = full_df[~full_df.index.isin(full_df[full_df.index.duplicated()].index)]

full_df.label.sum(), full_df.shape[0] - full_df.label.sum()
full_df = full_df.sample(frac=1)
final_df.sheet.unique()
final_df = df.merge(address_df, how='left', left_index=True, right_index=True)
final_df = final_df[final_df.sex.isin(['M', 'F']) & ~final_df.Age.isna()]

In [270]:
# negative_df = final_df[(final_df.label==1) & ~final_df.FE.isna()& ~final_df.FERRITIN.isna()]
# negative_df = final_df[(final_df.label==0) & (~(final_df.HBA1.isna() | final_df.HBA2.isna() |final_df.FE.isna() | final_df.FERRITIN.isna()))]
negative_df = full_df[(full_df.label==0)]
positive_df = full_df[(full_df.label==1)]
# full_df.index.duplicated()

In [271]:
p_df = positive_df[~((positive_df.MCV.astype(float)>=85) & (positive_df.MCH.astype(float)>=28))].iloc[-1000:]
t_1 = negative_df[((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))]
t_2 = negative_df[~((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))].iloc[-(1000-t_1.shape[0]):]
# negative_df.shape
negative_df[((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))].shape, t_1.shape, t_2.shape

((634, 14), (634, 14), (366, 14))

In [272]:
nm_df = pd.concat([t_1, t_2])
nm_df

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,FERRITIN,HBA1,HBA2,label
PID,,,,,,,,,,,,,,
1920034099,4.83,149.0,0.452,93.500000,31.000000,331.0,12.900000,314.0,6.88,552.500000,16.400000,97.500000,2.5,0.0
17007022,5.21,152.0,0.466,89.300003,29.200001,327.0,13.700000,295.0,8.29,379.399994,15.700000,97.900002,2.1,0.0
18020571,4.66,139.0,0.417,89.500000,29.799999,333.0,13.000000,306.0,9.18,402.700012,18.000000,97.199997,2.8,0.0
2020107252,4.75,146.0,0.435,91.699997,30.799999,336.0,13.300000,269.0,8.36,336.200012,22.000000,97.000000,3.0,0.0
1920008201,5.22,156.0,0.462,88.500000,29.900000,338.0,12.700000,239.0,6.57,326.799988,19.500000,97.500000,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18041715,4.91,143.0,0.407,83.000000,29.100000,350.0,13.300000,260.0,6.98,561.299988,12.700000,97.199997,2.8,0.0
18077352,4.16,84.0,0.279,67.199997,20.100000,300.0,16.799999,519.0,9.69,4.900000,3.300000,98.300003,1.7,0.0
1920069687,5.16,113.0,0.353,68.400002,21.900000,320.0,15.100000,244.0,8.27,95.400002,18.100000,95.599998,4.4,0.0


In [273]:
test_df = pd.concat([nm_df, p_df])
test_df = test_df.assign(train=0)
test_df.shape
test_address_df = final_df[final_df.index.isin(test_df.index)][['Age', 'sex', 'sheet']]
test_address_df  = test_address_df.reset_index().drop_duplicates(subset=['PID']).set_index('PID')

test_df = test_df.merge(test_address_df, how='left', left_index=True, right_index=True)


In [274]:
full_df2[~full_df2.index.isin(test_df.index)].shape, full_df2.shape, final_df.shape

((7513, 14), (9513, 14), (30888, 22))

In [275]:
train_df = final_df[~final_df.index.isin(test_df.index)]
train_df = train_df.assign(train=1)
train_df = train_df[
    (~((train_df.MCV.astype(float)>=85) & (train_df.MCH.astype(float)>=28)) & (train_df.label==1)) | (train_df.label==0)
]


In [276]:



splited_df = pd.concat([train_df, test_df])
# splited_df.assign(thalas=splited_df.label).to_csv('../datasets/splited_train_tes1.csv')
splited_df = splited_df.assign(sheet2=splited_df.sheet)

In [277]:
# (splited_df.sheet == '0504').sum()
# splited_df[splited_df.sheet.isin(['0504'])]['sheet'] = 'd2'
# splited_df[splited_df.sheet.isin(['normal', 'fe_normal', 'beta', 'fe_alpha', 'fe_beta', 'alpha_beta'])]['sheet'] = 'd1'
# splited_df[splited_df.sheet.isin(['1904', '2804', '2704', '2904'])]['sheet'] = 'd3'


In [278]:
splited_df.sheet.replace('0504', 'd2', inplace=True)
splited_df.sheet.replace('1904', 'd3', inplace=True)
splited_df.sheet.replace('2804', 'd3', inplace=True)
splited_df.sheet.replace('2704', 'd3', inplace=True)
splited_df.sheet.replace('2904', 'd3', inplace=True)
splited_df.sheet.replace('alpha', 'd1', inplace=True)
splited_df.sheet.replace('normal', 'd1', inplace=True)
splited_df.sheet.replace('fe_normal', 'd1', inplace=True)
splited_df.sheet.replace('beta', 'd1', inplace=True)
splited_df.sheet.replace('fe_alpha', 'd1', inplace=True)
splited_df.sheet.replace('fe_beta', 'd1', inplace=True)
splited_df.sheet.replace('alpha_beta', 'd1', inplace=True)


In [279]:
splited_df[splited_df.sheet.isin(['d2'])]['sheet']
splited_df.sheet.unique()

array(['d4', 'd3', 'd1', 'd2'], dtype=object)

In [297]:
splited_df.assign(thalas=splited_df.label).to_csv('../datasets/splited_train_test1.csv'), splited_df.shape

(None, (30442, 24))

In [300]:
bb_df = splited_df.assign(thalas=splited_df.label)
bb_df = bb_df.reset_index()[['PID', 'sheet2']]
bb_df.drop_duplicates(subset=['PID'])

,PID,sheet2
0,10000354,d4
8,10000366,d4
16,10000390,1904
18,10000546,2704
19,10000610,2804
...,...,...
30437,97221021,beta
30438,97200996,beta
30439,97380546,alpha
30440,19022565,d4


In [281]:
splited_df[splited_df.sheet.isin(['d1']) & splited_df.train.isin([0])]

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,...,HBF,sheet,inyear,label,alpha,beta,Age,sex,train,sheet2
PID,,,,,,,,,,,,,,,,,,,,,
97220959,5.15,158.0,0.454,88.2,30.700001,348.0,12.9,309.0,10.69,14.4,...,NaN,d1,NaN,0.0,NaN,NaN,1986,F,0,normal
97650122,4.72,137.0,0.437,92.6,29.000000,313.0,13.8,269.0,10.09,15.3,...,NaN,d1,NaN,0.0,NaN,NaN,1964,F,0,normal
97210264,5.03,141.0,0.429,85.3,28.000000,329.0,11.4,301.0,6.30,17.9,...,NaN,d1,NaN,0.0,NaN,NaN,2003,F,0,normal
97220844,4.39,137.0,0.424,96.7,31.299999,324.0,13.8,264.0,7.53,5.6,...,NaN,d1,NaN,0.0,NaN,NaN,1966,F,0,normal
97220897,4.48,130.0,0.411,91.6,28.900000,315.0,14.5,345.0,10.01,20.6,...,NaN,d1,NaN,0.0,NaN,NaN,1991,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96040769,6.31,132.0,0.418,66.3,20.900000,315.0,15.5,284.0,6.94,18.8,...,NaN,d1,NaN,1.0,NaN,NaN,1999,M,0,alpha
97221021,6.11,113.0,0.377,61.7,18.500000,299.0,16.2,328.0,11.10,18.9,...,NaN,d1,NaN,1.0,NaN,NaN,1982,F,0,beta
97200996,6.19,127.0,0.417,67.3,20.500000,304.0,15.1,220.0,6.41,26.3,...,NaN,d1,NaN,1.0,NaN,NaN,1982,M,0,beta


(25077, 24)

In [291]:
d = pd.read_csv('../datasets/splited_train_test.csv')
d

,PID,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,...,HBF,sheet,inyear,label,alpha,beta,Age,sex,train,thalas
0,10000354,3.96,89.0,0.286,72.200000,22.500000,311.0,26.3,554.0,10.24,...,NaN,d4,2015.0,1.0,0.0,1.0,1981,M,1,1.0
1,10000354,3.96,94.0,0.295,74.400000,23.700000,318.0,NaN,440.0,8.39,...,NaN,d4,2016.0,1.0,0.0,1.0,1981,M,1,1.0
2,10000354,3.30,74.0,0.246,74.500000,22.400000,300.0,25.6,547.0,16.00,...,NaN,d4,2017.0,1.0,0.0,1.0,1981,M,1,1.0
3,10000354,3.86,89.0,0.281,72.800000,22.900000,315.0,23.4,464.0,8.22,...,NaN,d4,2020.0,1.0,0.0,1.0,1981,M,1,1.0
4,10000354,3.72,87.0,0.292,78.400000,23.400000,298.0,22.5,503.0,9.23,...,NaN,d4,2018.0,1.0,0.0,1.0,1981,M,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30446,97630499,4.77,122.0,0.384,80.400002,25.600000,318.0,13.699999809265137,276.0,10.24,...,NaN,d1,NaN,1.0,NaN,NaN,2003,F,0,1.0
30447,97211296,4.70,131.0,0.393,83.599998,27.900000,333.0,10.899999618530273,318.0,6.70,...,NaN,d1,NaN,1.0,NaN,NaN,1989,F,0,1.0
30448,1920024736,5.45,110.0,0.355,65.000000,20.100000,309.0,16.5,247.0,9.88,...,NaN,d4,NaN,1.0,NaN,NaN,1991,F,0,1.0
30449,97212138,6.56,128.0,0.430,65.699997,19.600000,298.0,17.399999618530273,351.0,9.02,...,NaN,d1,NaN,1.0,NaN,NaN,2005,F,0,1.0


In [294]:
# d1_df = d[d.sheet.isin(['d1']) & d.sheet.isin([0])]
# d2_df = d[d.sheet.isin(['d2']) & d.sheet.isin([0])]
# d3_df = d[d.sheet.isin(['d3']) & d.sheet.isin([0])]
# d4_df = d[d.sheet.isin(['d4']) & d.sheet.isin([0])]
dic_df = {
    "d1": d[d.sheet.isin(['d1']) & d.train.isin([0])],
    "d2": d[d.sheet.isin(['d2']) & d.train.isin([0])],
    "d3": d[d.sheet.isin(['d3']) & d.train.isin([0])],
    "d4": d[d.sheet.isin(['d4']) & d.train.isin([0])]
    
}
# dic_df['d1']

In [296]:
for k, v in dic_df.items():
    print(k)
    print('positive: ', (v.label == 1).sum())
    print('negative: ', (v.label == 0).sum())
    print('------------------')

d1
positive:  561
negative:  159
------------------
d2
positive:  0
negative:  0
------------------
d3
positive:  101
negative:  83
------------------
d4
positive:  338
negative:  758
------------------


In [ ]:
pid_df.merge(tbm_df, how='left', left_index=True, right_index=True) # join left